In [1]:
import sys
from sys import platform
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from loader1 import *
from utils import *
from common import *
from experiment2 import *
# from sys import platform

import warnings
warnings.filterwarnings('ignore')

../Ride_Data_Cache/final_df_reg.msgpack exists
../Ride_Data_Cache/idle_times.msgpack exists


/Users/ashwin/Documents/Courses/Capstone/capstone_didi/experiment2.py:215: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  df_final, target_df = get_final_df_reg()


In [3]:
import time

Basic Coding 

In [4]:
df_final_no_decay, target_df_no_decay = get_final_df_reg(use_cache=False, decay='No Decay')

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
No Decay
../Ride_Data_Cache/active_times.msgpack exists
Decay Calculation done in 0.892963171005249
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.22311902046203613
1f
../Ride_Data_Cache/spatial_df.msgpack exists
spatial
Spatial Calculation done in 1.8458149433135986


In [5]:
df_final_old_decay, target_df_old_decay = get_final_df_reg(use_cache=False, decay='Old Decay')

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
Old Decay
Creating the exponential decay
Dumping to ../Ride_Data_Cache/idle_times_old.msgpack
Decay Calculation done in 49.72706389427185
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.19664788246154785
1f
../Ride_Data_Cache/spatial_df.msgpack exists
spatial
Spatial Calculation done in 1.5405511856079102


In [6]:
# Using the function create_modified_active_time_through_decay2 
# instead of create_modified_active_time_through_decay
df_final_new_decay1, target_df_new_decay1 = get_final_df_reg(use_cache=False, decay='New Decay', mult_factor=1)
df_final_new_decay10, target_df_new_decay10 = get_final_df_reg(use_cache=False, decay='New Decay', mult_factor=10)
df_final_new_decay100, target_df_new_decay100 = get_final_df_reg(use_cache=False, decay='New Decay', mult_factor=100)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
Creating the exponential decay
Dumping to ../Ride_Data_Cache/idle_times_new_1.msgpack
Decay Calculation done in 47.5724880695343
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.19587206840515137
1f
../Ride_Data_Cache/spatial_df.msgpack exists
spatial
Spatial Calculation done in 1.4727301597595215
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
Creating the exponential decay
Dumping to ../Ride_Data_Cache/idle_times_new_10.msgpack
Decay Calculation done in 48.01855397224426
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.20274782180786133
1f
../Ride_Data_Cache/spatial_df.msgpack exists
spatial
Spatial Calculation done in 1.7793340682983398
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
Creating the exponential decay
Dumping to ../Ride_Data_Cache/idle_times_new_100.msgpack
Decay Calcu

### Modelling Code 

We have till now coded the following models 

1. Linear Regression 
2. Linear Regression with L1 Regularization
3. Linear Regression with L2 Regularization
4. Random Forest Regressor 

Now we split the data in a ratio of 75:25

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor

In [8]:
def build_model(df, target_df, name, seed=0):
    # regression forest
    print(f'building regression model for {name}')
    X = df    
    xtrain, xtest, ytrain, ytest = train_test_split(X, target_df['target'], random_state=seed)

    sc = StandardScaler()

    xtrain_sc = sc.fit_transform(xtrain)

    rr = RandomForestRegressor(random_state=seed)
    rr.fit(xtrain_sc, ytrain)

    xtest_sc = sc.transform(xtest)
    
    print(rr.score(xtest_sc, ytest))
    
    return rr.score(xtest_sc, ytest)

In [9]:
df_list = [df_final_no_decay, df_final_old_decay, df_final_new_decay1, 
           df_final_new_decay10, df_final_new_decay100]
target_df_list = [target_df_no_decay, target_df_old_decay, target_df_new_decay1, 
                  target_df_new_decay10, target_df_new_decay100]
name_list = ['no_decay', 'old_decay', 'new_decay1', 'new_decay10', 'new_decay100']

In [10]:
result = {}
for df, target_df, name in zip(df_list, target_df_list, name_list):
    result[name] = build_model(df, target_df, name)

building regression model for no_decay
building regression model for old_decay
building regression model for new_decay1
building regression model for new_decay10
building regression model for new_decay100


In [11]:
result

{'no_decay': 0.446150277366595,
 'old_decay': 0.6245160896492661,
 'new_decay1': 0.591588937233497,
 'new_decay10': 0.668961645516281,
 'new_decay100': 0.7283817938885009}

In [12]:
df_final_old_decay.head()

,num_total_rides,num_pool_rides,% of pool rides,"(332903.267, 345801.219](3379575.462, 3389679.747]","(345801.219, 358571.469](3339057.28, 3349262.607]","(345801.219, 358571.469](3359366.892, 3369471.177]","(345801.219, 358571.469](3379575.462, 3389679.747]","(358571.469, 371341.718](3369471.177, 3379575.462]","(358571.469, 371341.718](3379575.462, 3389679.747]","(358571.469, 371341.718](3389679.747, 3399784.032]",...,"(435192.966, 447963.215](3369471.177, 3379575.462]","(435192.966, 447963.215](3379575.462, 3389679.747]","(435192.966, 447963.215](3389679.747, 3399784.032]","(435192.966, 447963.215](3399784.032, 3409888.316]","(435192.966, 447963.215](3409888.316, 3419992.601]","(447963.215, 460733.465](3359366.892, 3369471.177]","(447963.215, 460733.465](3379575.462, 3389679.747]","(447963.215, 460733.465](3389679.747, 3399784.032]","(447963.215, 460733.465](3409888.316, 3419992.601]",inactive_time
driver_id,,,,,,,,,,,,,,,,,,,,,
0000131d486b69eb77ab6e9e7cca9f4c,3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
00002724a19c5f6a54ae8d60a378997e,1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
000042288ba964aa04d14869bd56efb6,5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
000049d47b79747ad9927ba1bf5f4126,11,1.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,227.90122
00005adc52a78e1630cfb50e4bd0f89e,4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.99304


In [13]:
df_final_new_decay10.head()

,num_total_rides,num_pool_rides,% of pool rides,"(332903.267, 345801.219](3379575.462, 3389679.747]","(345801.219, 358571.469](3339057.28, 3349262.607]","(345801.219, 358571.469](3359366.892, 3369471.177]","(345801.219, 358571.469](3379575.462, 3389679.747]","(358571.469, 371341.718](3369471.177, 3379575.462]","(358571.469, 371341.718](3379575.462, 3389679.747]","(358571.469, 371341.718](3389679.747, 3399784.032]",...,"(435192.966, 447963.215](3369471.177, 3379575.462]","(435192.966, 447963.215](3379575.462, 3389679.747]","(435192.966, 447963.215](3389679.747, 3399784.032]","(435192.966, 447963.215](3399784.032, 3409888.316]","(435192.966, 447963.215](3409888.316, 3419992.601]","(447963.215, 460733.465](3359366.892, 3369471.177]","(447963.215, 460733.465](3379575.462, 3389679.747]","(447963.215, 460733.465](3389679.747, 3399784.032]","(447963.215, 460733.465](3409888.316, 3419992.601]",inactive_time
driver_id,,,,,,,,,,,,,,,,,,,,,
0000131d486b69eb77ab6e9e7cca9f4c,3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.931860
00002724a19c5f6a54ae8d60a378997e,1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
000042288ba964aa04d14869bd56efb6,5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.210168
000049d47b79747ad9927ba1bf5f4126,11,1.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,336.982874
00005adc52a78e1630cfb50e4bd0f89e,4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.410799
